<a href="https://colab.research.google.com/github/Zain1796/Lucky-Cement-AI-System/blob/main/NER_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets spacy seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=63a7a782194c9ea5334f3910dcd007253f27cdaf373cf7029e5b64ac81262509
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import json

def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
      data = []
      for line in file:
        if len(json.loads(line)["Comments"]) > 0:
          if ("discard" in json.loads(line)["Comments"][0].lower()) or len(json.loads(line)["label"]) < 1:
            continue
        data.append(json.loads(line))

        # data = [json.loads(line) for line in file if ("discard" not in json.loads(line)["Comments"][0].lower())]
    return data

data = load_data('/content/drive/MyDrive/Joyn_NewDataNLP/NER/all_NER_iter1.jsonl')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Joyn_NewDataNLP/NER/all_NER_iter1.jsonl'

In [ ]:
def create_label_map_from_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)

    label_map = {'O': 0}
    new_id = 1

    for entry in data:
        text = entry['text']
        # You can create a new ID here if needed
        label_id = new_id
        new_id += 1

        # Add the mapping to the label map
        label_map[text] = label_id

    return label_map

# Example usage:
filename = '/content/drive/MyDrive/Joyn_NewDataNLP/NER/label_config.json'
label_map = create_label_map_from_json(filename)
label_map

{'O': 0,
 'DG_Landed': 1,
 'DG_Retail': 2,
 'DG_Margin': 3,
 'DG_Sale Quantity': 4,
 'DG_Stock': 5,
 'DG_Supplier': 6,
 'Lucky_Landed': 7,
 'Lucky_Retail': 8,
 'Lucky_Margin': 9,
 'Lucky_Sale Quantity': 10,
 'Lucky_Stock': 11,
 'Lucky_Supplier': 12,
 'Thatta_Landed': 13,
 'Thatta_Retail': 14,
 'Thatta_Margin': 15,
 'Thatta_Sale Quantity': 16,
 'Thatta_Stock': 17,
 'Thatta_Supplier': 18,
 'PakLand_Landed': 19,
 'PakLand_Retail': 20,
 'PakLand_Margin': 21,
 'PakLand_Sale Quantity': 22,
 'PakLand_Stock': 23,
 'PakLand_Supplier': 24,
 'Power_Landed': 25,
 'Power_Retail': 26,
 'Power_Margin': 27,
 'Power_Sale Quantity': 28,
 'Power_Stock': 29,
 'Power_Supplier': 30,
 'Falcon_Landed': 31,
 'Falcon_Retail': 32,
 'Falcon_Margin': 33,
 'Falcon_Sale Quantity': 34,
 'Falcon_Stock': 35,
 'Falcon_Supplier': 36,
 'Other_Landed': 37,
 'Other_Retail': 38,
 'Other_Margin': 39,
 'Other_Sale Quantity': 40,
 'Other_Stock': 41,
 'Running_Scheme': 42,
 'Other_Supplier': 43,
 'Payment_Method': 44,
 'Transpor

In [ ]:
len(data)

342

In [ ]:
import nltk
nltk.download('punkt')

def custom_word_tokenize_with_indices(text):
    words = nltk.word_tokenize(text)
    indices = []
    start = 0

    for word in words:
        start = text.find(word, start)
        end = start + len(word)
        indices.append((start, end))
        start = end

    return words, indices

# Example usage
text = "This is a sample sentence sample this."
words, indices = custom_word_tokenize_with_indices(text)

list(zip(words, indices))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[('This', (0, 4)),
 ('is', (5, 7)),
 ('a', (8, 9)),
 ('sample', (10, 16)),
 ('sentence', (17, 25)),
 ('sample', (26, 32)),
 ('this', (33, 37)),
 ('.', (37, 38))]

In [ ]:
import nltk

# Download the Urdu tokenizer from nltk
nltk.download('punkt')

def convert_to_listed_form(json_data):
# Tokenize the text using nltk
  # tokens = list(json_data["text"])
  tokens = nltk.word_tokenize(json_data["text"])
  text = json_data["text"]
  # Initialize labels with 'O' (Outside) for all tokens
  labels = [0] * len(tokens)
  indices = []
  start = 0
  for word in tokens:
      start = text.find(word, start)
      end = start + len(word)
      indices.append((start, end))
      start = end

  # Process label spans
  for span in json_data["label"]:
      start_l, end_l, entity = span
      # print("span ", span)
      for i in range(len(tokens)):
          # print(indices[i])
          if start_l <= indices[i][0] and end_l >= indices[i][1]:
              # print("happened")
              labels[i] = label_map[entity]

  # Create the output list of tokens and labels
  output = (tokens, labels)
  return output

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def make_dataset(data):
    whole_dataset = []
    for i, d in enumerate(data):
      temp_dict = {}
      # print(d["Comments"])
      print(i, d["label"])
      temp_dict["id"] = i
      temp_dict["tokens"], temp_dict["ner_tags"] = convert_to_listed_form(d)
      whole_dataset.append(temp_dict)
    return whole_dataset

data_converted = make_dataset(data)

0 [[51, 55, 'Lucky_Landed'], [69, 73, 'Lucky_Landed'], [100, 111, 'Lucky_Retail'], [160, 164, 'Falcon_Landed'], [188, 204, 'Falcon_Retail'], [280, 285, 'Lucky_Supplier'], [311, 330, 'Falcon_Supplier']]
1 [[35, 42, 'Lucky_Supplier'], [67, 71, 'Lucky_Landed'], [103, 107, 'Lucky_Retail'], [143, 155, 'Falcon_Supplier'], [166, 170, 'Falcon_Landed'], [178, 182, 'Falcon_Retail']]
2 [[97, 102, 'Lucky_Landed'], [137, 142, 'Lucky_Retail'], [174, 179, 'Falcon_Landed'], [211, 216, 'Falcon_Retail'], [220, 225, 'Falcon_Retail'], [297, 308, 'Falcon_Supplier'], [297, 308, 'Lucky_Supplier']]
3 [[407, 411, 'Falcon_Landed'], [3462, 3466, 'Falcon_Stock']]
4 [[798, 807, 'Falcon_Retail']]
5 [[422, 426, 'Power_Landed'], [429, 433, 'Power_Landed'], [477, 481, 'DG_Landed'], [490, 494, 'DG_Landed'], [673, 677, 'Other_Retail'], [801, 804, 'Other_Margin']]
6 [[617, 621, 'Falcon_Retail'], [622, 626, 'Falcon_Retail'], [1118, 1122, 'Falcon_Landed'], [1124, 1128, 'Falcon_Landed']]
7 [[474, 483, 'Lucky_Stock']]
8 [[30

In [ ]:
import numpy as np
np.unique([len(d["tokens"]) for d in data_converted], return_counts=True)
data_converted[0]["ner_tags"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 31,
 0,
 0,
 0,
 0,
 0,
 0,
 32,
 32,
 32,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 36,
 36,
 36,
 36,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset, load_metric
import pandas as pd
import random

random.shuffle(data_converted)
df = pd.DataFrame.from_dict(data_converted)

datasets = DatasetDict({
    'train': Dataset.from_pandas(df.iloc[:int(len(df) * 0.90)]),
    'test': Dataset.from_pandas(df.iloc[int(len(df) * 0.90):])
})

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 307
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 35
    })
})

In [ ]:
label_list = list(label_map.keys()) + ["O"]
label_list

['O',
 'DG_Landed',
 'DG_Retail',
 'DG_Margin',
 'DG_Sale Quantity',
 'DG_Stock',
 'DG_Supplier',
 'Lucky_Landed',
 'Lucky_Retail',
 'Lucky_Margin',
 'Lucky_Sale Quantity',
 'Lucky_Stock',
 'Lucky_Supplier',
 'Thatta_Landed',
 'Thatta_Retail',
 'Thatta_Margin',
 'Thatta_Sale Quantity',
 'Thatta_Stock',
 'Thatta_Supplier',
 'PakLand_Landed',
 'PakLand_Retail',
 'PakLand_Margin',
 'PakLand_Sale Quantity',
 'PakLand_Stock',
 'PakLand_Supplier',
 'Power_Landed',
 'Power_Retail',
 'Power_Margin',
 'Power_Sale Quantity',
 'Power_Stock',
 'Power_Supplier',
 'Falcon_Landed',
 'Falcon_Retail',
 'Falcon_Margin',
 'Falcon_Sale Quantity',
 'Falcon_Stock',
 'Falcon_Supplier',
 'Other_Landed',
 'Other_Retail',
 'Other_Margin',
 'Other_Sale Quantity',
 'Other_Stock',
 'Running_Scheme',
 'Other_Supplier',
 'Payment_Method',
 'Transport_per_bag',
 'DG_DO Rate',
 'DG_Warehouse Rate',
 'Transport_otherwise',
 'Other_DO Rate',
 'Other_Warehouse Rate',
 'Thatta_DO Rate',
 'Thatta_Warehouse Rate',
 'PakLand

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,304,"[نومن, سیمنٹ, جام, روڈ, لکی, لینڈڈ, 1070, ریٹیل, 1100, ڈی, جی, لینڈڈ, 1070, ریٹیل, 1100, فالکن, لینڈڈ, 1090, ریٹیل, 1120, ٹھٹہ, لینڈڈ, 1010, ریٹیل, 1140, پاک, لینڈ, لینڈڈ, 1020, ریٹیل, 1050]","[0, 0, 0, 0, 0, 0, 7, 0, 8, 0, 0, 0, 1, 0, 2, 0, 0, 31, 0, 32, 0, 0, 13, 0, 14, 0, 0, 0, 19, 0, 20]"
1,156,"[آپ, کے, پاس, تو, یہ, سیمنٹ, بھی, تقریباً, 20-30, رہے, ہیں, 30-30, بوری, ڈیلی, آرہا, ہے, اچھا, ڈیلی, منگوا, رہے, ہیں, ڈیلی, 100, بیٹھ, 100, بیٹھ, ماشاللہ, 100, بیٹھ, تو, نکل, ہی, رہے, ہیں, نا, کل, کا, 4-5, ہم, کافا, منگا, رہے, ہیں, پاک, لینڈ, تو, یہ, پہلے, آپ, گاڑی, منگاتے, تھے, تو, وہ, اتر, سے, رہے, تھے, اب, کیسے, لے, رہے, ہیں, عبدل, رحمن, اور, فالکن, نثار, بھائی, نہیں, منگا, رہے, وہی, پرانے, ریٹ, نہ, پڑا, ہوا, ہے, اچھا, ورے, بھائی, پاک, لینڈ, میں, پڑا, ہے, -, ?, ?, -, 50, بوری, منگوائی, ہے, انہوں, نے, 1020, کے, حساب, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2,225,"[اور, بتائیں, سیمنٹ, کا, کوئی, ارڈر, وغیرہ, ہے, آپکا, #, سیمنٹ, کل, آیا, ہے, #, کل, آگیا, ہے, لکی, کا, لکی, کا, کتنا, بوری, آگیا, #, 100, آئ, 200, ادھر, آیا, 100, اامر, کے, پاس, #, اامر, کے, پاس, چلو, بھی, اامر, کے, پاس, بھی, جا, رہے, ہیں, اور, ابھی, کتنا, 10, بوری, کا, چال, 8, بوری, کا, آج, چال, نہیں, لگا, رہے, #, 5, بوری, کا, چال, ہے, #, میں, آج, جا, رہا, ہوں, تو, #, نہیں, یار, #, 4, 5, دن, کے, لئے, جا, رہا, ہوں, اچھا, اچھا, شادی, اور, باقی, پاک, لینڈ, #, ابھی, صحیح, ہوگیا, نا, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
3,188,"[شکر, الحمدللہ, #, کام, وام, کا, سنائیں, شوقت, بھائی, کیسا, جا, رہا, ہے, #, کام, وام, ابھی, بس, ایسا, ہی, ہے, #, کیوں, کیا, ہوا, ابھی, تو, کچھ, بہتر, ہوگیا, ہے, ماشااللہ, پہلے, سے, #, بہتر, کدھر, ہوا, ہے, #, بہتر, ہوگیا, تو, آپ, کی, گاڑی, لوڈ, ہوئی, ہے, #, وہ, تو, ہے, #, اچھا, الله, کا, کرم, ہے, آپ, سنائیں, ٹھیک, ہیں, طبیعت, ٹھیک, ہے, خیریت, ہے, مجھے, پہچانا, -, ?, ?, -, میرا, شروع, سے, میں, پلی, پور, یہ, ڈی, جی, اور, ابھی, یہ, تقریبن, 1-2, سال, ہوگے, پاور, چلا, رہا, ہے, #, اچھا, 1, ارڈر, کرایا, تو, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
4,301,"[لکی, فرمان, سٹیل, بکرہ, پوڑی, لکی, لینڈڈ, ریٹ, 1065, ریٹیل, 1090, فالکن, لینڈڈ, 1085, ریٹیل, 1120, پاور, لینڈڈ, 1060, ریٹیل, 1090, ڈی, جی, لینڈڈ, 1060, ریٹیل, 1090, پاک, لینڈ, لینڈڈ, 1010, ریٹیل, 1040]","[0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 8, 0, 0, 31, 0, 32, 0, 0, 25, 0, 26, 0, 0, 0, 1, 0, 2, 0, 0, 0, 19, 0, 20]"


In [ ]:
task = "ner"
label_all_tokens = True
model_checkpoint = "mwz/UrduNER"
batch_size = 8

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.54M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, max_length=512, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# datasets.features[f"ner_tags"] = Sequence(feature=ClassLabel(names=label_list))
# datasets = datasets.cast_column("labels", Sequence(ClassLabel(names=label_list)))

In [ ]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)

In [ ]:
# tokenized_datasets = DatasetDict({
#     "train": Dataset.from_dict(tokenize_and_align_labels(datasets['train'])),
# # tokenized_datasets["train"] = tokenize_and_align_labels(datasets['train'])
#     "test": Dataset.from_dict(tokenize_and_align_labels(datasets['test']))
#     })
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/307 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 307
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 35
    })
})

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at mwz/UrduNER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([62]) in the model instantiated
- classifier.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([62, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"/content/model{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

<ipython-input-27-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
labels = [label_list[i] for i in datasets["train"][5][f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Other_Landed seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Falcon_Landed seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'alcon_Landed': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.135683,0.000000,0.000000,0.000000,0.977634
2,No log,0.119364,0.000000,0.000000,0.000000,0.977634
3,No log,0.118150,0.000000,0.000000,0.000000,0.977634


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Lucky_Stock seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DG_Landed seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Falcon_Supplier seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Lucky_Landed seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DG_Supplier seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/se

TrainOutput(global_step=117, training_loss=0.23929798093616453, metrics={'train_runtime': 91.6111, 'train_samples_per_second': 10.053, 'train_steps_per_second': 1.277, 'total_flos': 236951617545060.0, 'train_loss': 0.23929798093616453, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.11814957857131958,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.9776335409174634,
 'eval_runtime': 1.4061,
 'eval_samples_per_second': 24.892,
 'eval_steps_per_second': 3.556,
 'epoch': 3.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'G_Landed': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7},
 'G_Retail': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 3},
 'G_Supplier': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 3},
 'akLand_Landed': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7},
 'akLand_Margin': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 4},
 'akLand_Retail': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8},
 'akLand_Supplier': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 3},
 'alcon_Landed': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 14},
 'alcon_Margin': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7},
 'alcon_Retail': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 12},
 'alcon_Stock': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2},
 'alcon_Supplier': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7},
 'hatta_Landed': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2},
 'hatta_Retai

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[0, 377, 2409, 657, 316, 405, 11497, 304, 330, 381, 1617, 469, 12604, 316, 2563, 444, 326, 2563, 444, 292, 1000, 316, 1433, 325, 2738, 8130, 418, 445, 340, 330, 313, 340, 292, 1000, 316, 309, 657, 445, 1005, 428, 322, 340, 326, 3914, 1153, 358, 377, 2574, 269, 32636, 11497, 400, 279, 1333, 261, 620, 768, 316, 11497, 3008, 309, 500, 358, 32636, 19137, 19137, 333, 32636, 1570, 3247, 2575, 604, 2370, 500, 358, 377, 2773, 657, 316, 933, 430, 1033, 5230, 316, 418, 4308, 299, 4699, 1936, 316, 32636, 6586, 4699, 2894, 316, 340, 333, 32636, 8942, 2894, 316, 409, 660, 1596, 424, 692, 2894, 316, 692, 2894, 316, 418, 3744, 4111, 316, 432, 11836, 2320, 3744, 316, 430, 692, 28314, 316, 10885, 316, 432, 28314, 316, 1433, 6195, 28314, 316, 1433, 1434, 6946, 6195, 418, 3173, 333, 316, 9553, 333, 316, 508, 269, 432, 418, 2903, 333, 316, 3101, 432, 430, 1315, 445, 669, 15699, 432, 370, 445, 2788, 2788, 607, 6586, 2067, 283, 309, 667, 514, 430, 833, 1315, 445, 1433, 432, 430, 1329, 304, 81